In [ ]:
!pip install pennylane
!pip install pennylane-qiskit

In [ ]:
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

In [ ]:
df = pd.read_csv("DatasetExos.csv" ,delimiter="," )
df = df.sample(n=100)
df

,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Exercise,Category,Set
1028,2019-01-11 15:56:00.200,-0.207000,0.680500,-0.176500,27.8170,-5.9390,-44.1340,B,ohp,medium,37.0
1489,2019-01-11 16:09:38.200,-0.283500,0.886000,0.487000,-15.1098,3.9512,3.8534,B,squat,medium,26.0
789,2019-01-11 15:48:57.400,-0.349500,1.221000,0.046000,-36.1584,-24.4268,17.1462,B,ohp,medium,84.0
2494,2019-01-14 13:32:14.000,-0.050000,0.700500,-0.073000,2.7196,3.1220,4.8168,C,bench,heavy,72.0
4409,2019-01-15 19:17:38.200,0.429333,0.609000,0.685000,9.2320,-7.3538,-2.3172,C,squat,heavy,64.0
...,...,...,...,...,...,...,...,...,...,...,...
8976,2019-01-20 17:33:21.400,-0.024500,-0.941500,-0.048500,-0.4634,-1.1828,3.4756,E,row,medium,40.0
8532,2019-01-20 17:22:45.400,0.766000,-0.297500,0.392500,-55.1950,-88.6464,-24.7682,E,rest,sitting,89.0
956,2019-01-11 15:54:08.000,-0.188333,0.713667,-0.184000,28.0122,-4.5856,-38.4636,A,ohp,medium,56.0
7220,2019-01-18 17:33:13.600,0.025000,-0.629333,0.160333,-17.4268,9.3170,-4.3780,D,row,medium,57.0


In [ ]:
df.columns

Index(['ep (ms)', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z',
       'ID', 'Exercise', 'Category', 'Set'],
      dtype='object')

In [ ]:
df = df.drop(columns=["ep (ms)" , "Exercise"  , "ID" ])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
1028,-0.207000,0.680500,-0.176500,27.8170,-5.9390,-44.1340,medium,37.0
1489,-0.283500,0.886000,0.487000,-15.1098,3.9512,3.8534,medium,26.0
789,-0.349500,1.221000,0.046000,-36.1584,-24.4268,17.1462,medium,84.0
2494,-0.050000,0.700500,-0.073000,2.7196,3.1220,4.8168,heavy,72.0
4409,0.429333,0.609000,0.685000,9.2320,-7.3538,-2.3172,heavy,64.0
...,...,...,...,...,...,...,...,...
8976,-0.024500,-0.941500,-0.048500,-0.4634,-1.1828,3.4756,medium,40.0
8532,0.766000,-0.297500,0.392500,-55.1950,-88.6464,-24.7682,sitting,89.0
956,-0.188333,0.713667,-0.184000,28.0122,-4.5856,-38.4636,medium,56.0
7220,0.025000,-0.629333,0.160333,-17.4268,9.3170,-4.3780,medium,57.0


In [ ]:
for i in df.columns:
    if i not in ["Category"]:
        df[i] = pd.to_numeric(df[i])
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
1028,-0.207000,0.680500,-0.176500,27.8170,-5.9390,-44.1340,medium,37.0
1489,-0.283500,0.886000,0.487000,-15.1098,3.9512,3.8534,medium,26.0
789,-0.349500,1.221000,0.046000,-36.1584,-24.4268,17.1462,medium,84.0
2494,-0.050000,0.700500,-0.073000,2.7196,3.1220,4.8168,heavy,72.0
4409,0.429333,0.609000,0.685000,9.2320,-7.3538,-2.3172,heavy,64.0
...,...,...,...,...,...,...,...,...
8976,-0.024500,-0.941500,-0.048500,-0.4634,-1.1828,3.4756,medium,40.0
8532,0.766000,-0.297500,0.392500,-55.1950,-88.6464,-24.7682,sitting,89.0
956,-0.188333,0.713667,-0.184000,28.0122,-4.5856,-38.4636,medium,56.0
7220,0.025000,-0.629333,0.160333,-17.4268,9.3170,-4.3780,medium,57.0


In [ ]:
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     100 non-null    int64  
 1   Acc_x     100 non-null    float64
 2   Acc_y     100 non-null    float64
 3   Acc_z     100 non-null    float64
 4   Gyro_x    100 non-null    float64
 5   Gyro_y    100 non-null    float64
 6   Gyro_z    100 non-null    float64
 7   Category  100 non-null    object 
 8   Set       100 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 7.2+ KB


In [ ]:
df = df.dropna()
df = df.drop(columns=["index"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Acc_x     100 non-null    float64
 1   Acc_y     100 non-null    float64
 2   Acc_z     100 non-null    float64
 3   Gyro_x    100 non-null    float64
 4   Gyro_y    100 non-null    float64
 5   Gyro_z    100 non-null    float64
 6   Category  100 non-null    object 
 7   Set       100 non-null    float64
dtypes: float64(7), object(1)
memory usage: 6.4+ KB


In [ ]:
df

,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,Category,Set
0,-0.207000,0.680500,-0.176500,27.8170,-5.9390,-44.1340,medium,37.0
1,-0.283500,0.886000,0.487000,-15.1098,3.9512,3.8534,medium,26.0
2,-0.349500,1.221000,0.046000,-36.1584,-24.4268,17.1462,medium,84.0
3,-0.050000,0.700500,-0.073000,2.7196,3.1220,4.8168,heavy,72.0
4,0.429333,0.609000,0.685000,9.2320,-7.3538,-2.3172,heavy,64.0
...,...,...,...,...,...,...,...,...
95,-0.024500,-0.941500,-0.048500,-0.4634,-1.1828,3.4756,medium,40.0
96,0.766000,-0.297500,0.392500,-55.1950,-88.6464,-24.7682,sitting,89.0
97,-0.188333,0.713667,-0.184000,28.0122,-4.5856,-38.4636,medium,56.0
98,0.025000,-0.629333,0.160333,-17.4268,9.3170,-4.3780,medium,57.0


In [ ]:
from sklearn.preprocessing import StandardScaler

Y = df["Category"]
X = df.drop(columns="Category")
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train

array([[-6.58482873e-02, -1.31580208e+00, -8.62482619e-01,
         4.04281222e-01,  1.04013608e-01, -2.78891963e-01,
         2.36546035e-01],
       [-1.10362654e+00,  1.17359902e+00, -7.21001692e-01,
        -8.49252705e-01, -3.77771057e-01,  1.66999651e+00,
        -1.09118651e+00],
       [ 5.17258039e-02,  6.98558027e-01,  1.62509956e+00,
         7.29656826e-01, -2.48964098e-01,  1.58984465e-01,
        -6.71902548e-01],
       [-1.54130119e+00,  9.83210042e-01, -7.99436732e-01,
        -1.14303773e+00, -1.27024279e+00,  1.86148833e+00,
         1.80886089e+00],
       [-1.19537511e+00,  9.62718078e-01, -9.90560088e-01,
        -4.03220681e-01, -6.79719808e-01,  8.97420169e-01,
        -1.09118651e+00],
       [ 1.55640232e+00,  6.69869277e-01,  1.61119968e+00,
         4.40384745e-01, -2.48127333e-01, -2.12850227e-01,
         7.95591317e-01],
       [ 3.34447318e-01,  8.12195285e-01,  1.34015201e+00,
        -1.93433688e-01,  4.71410149e-02,  1.39668535e-01,
        -1.1261268

In [ ]:
X_train.shape

(90, 7)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_train_encoded

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 3, 0, 0, 3, 1, 3, 2, 1, 1, 3, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 3, 0, 1, 1, 3, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       2, 0, 2, 0, 3, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0])

In [ ]:
import pennylane as qml
import numpy as np
from sklearn.preprocessing import OneHotEncoder
dev = qml.device('qiskit.aer', wires=7)

@qml.qnode(dev)
def quantum_circuit(weights, x):
    qml.BasisState(x, wires=[0, 1, 2, 3, 4, 5, 6])

    qml.Rot(weights[0], weights[1], weights[2], wires=0)
    qml.Rot(weights[3], weights[4], weights[5], wires=1)
    qml.Rot(weights[6], weights[7], weights[8], wires=2)
    qml.Rot(weights[9], weights[10], weights[11], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])

    return qml.expval(qml.PauliZ(1))

def cost(weights, X, Y):
    predictions = [quantum_circuit(weights, x) for x in X]
    return np.mean((np.array(predictions) - Y) ** 2)

X_train_binary = (X_train > 0.5).astype(int)

#random weights
weights = np.random.rand(12) * 2 * np.pi
learning_rate = 0.1
for epoch in range(50):
    #simulating the backpropagation since we don't have a predefined way to do so
    grad = np.zeros_like(weights)
    for i in range(len(weights)):
        weights[i] += 0.01
        loss_plus = cost(weights, X_train_binary, y_train_encoded)
        weights[i] -= 0.02
        loss_minus = cost(weights, X_train_binary, y_train_encoded)
        grad[i] = (loss_plus - loss_minus) / 0.02
        weights[i] += 0.01

    weights -= learning_rate * grad
    if epoch % 10 == 0:
        print(f"epoch {epoch}: loss = {cost(weights, X_train_binary, y_train_encoded)}")

print("Trained weights:", weights)

epoch 0: loss = 1.485504150390625
epoch 10: loss = 1.3625911288791233
epoch 20: loss = 1.4311317443847655
epoch 30: loss = 1.3894468095567492
epoch 40: loss = 1.401260503133138
Trained weights: [2.08805729 1.93808015 0.46775187 1.48591987 1.52581423 3.4469708
 3.83360113 5.10953411 2.0241484  4.76507761 5.00266643 6.51990449]
